In [0]:
import os
import sys
import numpy as np
import json
import nltk
import pandas as pd
import csv
import random
import logging
import tensorflow as tf
from collections import Counter
import pathlib
import pickle
import progressbar

#from tensorflow.python.keras.utils import Progbar

sys.path.append(os.getcwd() + "/bert")
from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

from text_preprocessing import tokenizer_word
from language_model_processing import read_raw_data_preprocess_and_save, create_vocab_df
from bpe import create_token_vocabulary, get_stats, merge_vocab, Encoder

c:\users\usherwoodpe\appdata\local\continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [0]:
DATASET_NAME = 'wiki_pt'
INPUT_TYPE = 'txt' #Options: tokens, txt, csv
TO_SPLIT_CLITICS = True #Set to false if clitics already tokenized
DATASET_FILE_MAP = {'all': 'pt_wiki_corpus_v1_0_1.txt'}

if DATASET_FILE_MAP.get('all'):
    file_split = 'all'
else:
    file_split = 'split'

UNK_TOKEN = None #none if isnt one
SEQUENCE_LENGTH = 20
NUM_MERGES = 10000 #VOCABULARY_SIZE = NUM_MERGES + N_BYTES (~1500)

mini_batch_size = 64
N_SPLITS = 10 #of training data on disk, this is purely a question of resource efficiency

In [0]:
notebook_dir = pathlib.Path.cwd()
repo_dir = notebook_dir.parent
(repo_dir / "models").mkdir(exist_ok = True)
(repo_dir / "models" / "base").mkdir(exist_ok = True)
dataset_dir = repo_dir / "datasets" / "base" / DATASET_NAME
models_dir = repo_dir / "models" / "base"
(models_dir / DATASET_NAME).mkdir(exist_ok = True)
(models_dir / DATASET_NAME / "preprocessed_base_data").mkdir(exist_ok = True)
(models_dir / DATASET_NAME / "processed_base_data").mkdir(exist_ok = True)
(models_dir / DATASET_NAME / "processed_base_data" / "train").mkdir(exist_ok = True)
(models_dir / DATASET_NAME / "processed_base_data" / "validate").mkdir(exist_ok = True)
(models_dir / DATASET_NAME / "pretraining_base_data").mkdir(exist_ok = True)
processed_data_dir = models_dir / DATASET_NAME / "processed_base_data"
pretraining_data_dir = models_dir / DATASET_NAME / "pretraining_base_data"
(models_dir / DATASET_NAME / "language_maps").mkdir(exist_ok = True)
language_maps_dir = repo_dir / "models" / "base" / DATASET_NAME / "language_maps"
(language_maps_dir).mkdir(exist_ok = True)
    
models_dir = models_dir / DATASET_NAME

def save_obj(obj, directory, name):
    with open(directory / "{}.pkl".format(name), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name, directory):
    with open(os.path.join(directory, name + '.pkl'), 'rb') as f:
        return pickle.load(f)

# 1. Clean text and build tokenizer

In [0]:
read_raw_data_preprocess_and_save(dataset_file_map=DATASET_FILE_MAP, 
                                  models_dir=models_dir, 
                                  dataset_dir=dataset_dir,
                                  input_type=INPUT_TYPE,
                                  split_clitics=TO_SPLIT_CLITICS,
                                  remove_numbers=False,
                                  base_folder='preprocessed_base_data')

In [0]:
if file_split == 'all':
    preprocessed_training_data_dir = repo_dir / "models" / "base" / DATASET_NAME / "preprocessed_base_data"
else:
    preprocessed_training_data_dir = repo_dir / "models" / "base" / DATASET_NAME / "preprocessed_base_data" / "train"

corpus = []
MAX = 10
if not MAX:
    MAX = len(os.listdir(preprocessed_training_data_dir))

with progressbar.ProgressBar(max_value=MAX) as bar:
    for i, file in enumerate(os.listdir(preprocessed_training_data_dir)):
        with open(os.path.join(preprocessed_training_data_dir, file), 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            words = list(reader)[0]
            corpus += words
        if i == MAX:
            break
        bar.update(i)
    
len(corpus)

100% (10 of 10) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


1087404

In [0]:
# U+E000..U+F8FF is defined as a private use area so we use for space and unk
unk = '[UNK]'
spc = chr(int("E001", 16))
cls = '[CLS]'
sep = '[SEP]'
mask = '[MASK]'
pad = '[PAD]'

id_to_vocab = create_token_vocabulary()
unk_id = len(id_to_vocab)
spc_id = len(id_to_vocab) + 1
cls_id = len(id_to_vocab) + 2
sep_id = len(id_to_vocab) + 3
mask_id = len(id_to_vocab) + 4
pad_id = len(id_to_vocab) + 5

id_to_vocab[unk_id] = unk
id_to_vocab[spc_id] = spc
id_to_vocab[cls_id] = cls
id_to_vocab[sep_id] = sep
id_to_vocab[mask_id] = mask
id_to_vocab[pad_id] = pad

save_obj(id_to_vocab, language_maps_dir, "byte_decoder")

In [0]:
vocab_to_id = {v: i for i, v in id_to_vocab.items()}
id_to_vocab = {i: v for v, i in vocab_to_id.items()}  # Reverse as the emoji and other characters have some overlap 
_ = vocab_to_id.pop(unk)

print("BPE vocab size:", len(vocab_to_id))

corpus = tokenizer_word(corpus,
                        keep_phrases=False,
                        tokenize_punc=True,
                        split_clitics=True,
                        keep_preceeding_space=True)

corpus_ids = [[vocab_to_id.get(l, unk_id) if l is not " " else spc_id for l in word] for word in corpus]
corpus = [" ".join([id_to_vocab[l] for l in word]) for word in corpus_ids]

count_dict = Counter(corpus)
vocab_df = pd.DataFrame(np.array([list(dict(count_dict).keys()), list(dict(count_dict).values())]).T,
                        columns=['Word', 'Freq'])
vocab_df['Freq'] = vocab_df['Freq'].astype(np.float64)
print("Total word vocab size", len(vocab_df))
df_dict = vocab_df.sort_values(by=['Freq'], ascending=False).to_dict("records")
df_dict = {item["Word"]: item["Freq"] for item in df_dict}

bpe_merges = []
vocab_to_id_current_max_id = sorted(list(vocab_to_id.values()))[-1]
with progressbar.ProgressBar(max_value=NUM_MERGES) as bar:
    for i in range(NUM_MERGES):
        vocab_to_id_current_max_id += 1
        pairs = get_stats(df_dict)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        bpe_merges.append(best)
        vocab_to_id["".join(best)] = vocab_to_id_current_max_id
        df_dict = merge_vocab(best, df_dict)
        bar.update(i)
id_to_vocab = {i: v for v, i in vocab_to_id.items()}
id_to_vocab[unk_id] = unk

save_obj(bpe_merges, language_maps_dir, "bpe_merges")
save_obj(id_to_vocab, language_maps_dir, "id_to_vocab")
save_obj(vocab_to_id, language_maps_dir, "vocab_to_id")
pd.DataFrame(list(vocab_to_id.keys())).to_csv(language_maps_dir / 'vocab_file.csv', encoding='utf-8', header=False, index=False, quoting=csv.QUOTE_NONE, escapechar='\\')

In [0]:
testcase = " ".join(words[1000:1020])
bert_tokenizer = tokenization.FullTokenizer(language_maps_dir)
print(testcase)
print(bert_tokenizer.tokenize(testcase))

- a localizada na direcção da constelação de cetus . possui uma declinação de - 10 01 ' 18 e
['-', 'a', 'localizada', 'na', 'direcção', 'da', 'constelação', 'de', 'cetus', '.', 'possui', 'uma', 'declinação', 'de', '-', '10', '01', "'", '18', 'e']


In [0]:
testcase = "Olá isso é mais uma BAGUNCA 😂😂😂"
print(testcase)
print(bert_tokenizer.tokenize(testcase))

Olá isso é mais uma BAGUNCA 😂😂😂
['ol', 'á', 'isso', 'é', 'mais', 'uma', 'ba', 'gun', 'ca', '😂', '😂', '😂']


# Prep data file

In [0]:
if file_split == 'all':
    preprocessed_training_data_dir = repo_dir / "models" / "base" / DATASET_NAME / "preprocessed_base_data"
else:
    preprocessed_training_data_dir = repo_dir / "models" / "base" / DATASET_NAME / "preprocessed_base_data" / "train"

corpus = []
MAX = 1
if not MAX:
    MAX = len(os.listdir(preprocessed_training_data_dir))

with progressbar.ProgressBar(max_value=MAX) as bar:
    for i, file in enumerate(os.listdir(preprocessed_training_data_dir)):
        with open(os.path.join(preprocessed_training_data_dir, file), 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            words = list(reader)[0]
            corpus += words
        if i == MAX:
            break
        bar.update(i)
    


100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [0]:
x[:3000]

'. astronomia é uma ciência natural que estuda corpos celestes como estrelas, planetas, cometas, nebulosas, aglomerados de estrelas, galáxias e fenômenos que se originam fora da atmosfera da terra como a radiação cósmica de fundo em micro - ondas. preocupada com a evolução, a física, a química e o movimento de objetos celestes, bem como a formação e o desenvolvimento do universo. \n a astronomia é uma das mais antigas ciências. culturas pré - históricas deixaram registrados vários artefatos astronômicos, como stonehenge, os montes de newgrange e os menires. as primeiras civilizações, como os babilônios, gregos, chineses, indianos, iranianos e maias realizaram observações metódicas do céu noturno. no entanto, a invenção do telescópio permitiu o desenvolvimento da astronomia moderna. historicamente, a astronomia incluiu disciplinas tão diversas como astrometria, navegação astronômica, astronomia observacional e a elaboração de calendários. durante o período medieval, seu estudo era obrig

In [0]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import sent_tokenize
corp_str = TreebankWordDetokenizer().detokenize(corpus).replace(' .', '.')
for i in range(10):
    corp_str = corp_str.replace('. '+str(i), '.'+str(i))

corp_list = [x for x in sent_tokenize(corp_str) if x != '.']

In [0]:
corp_list[:31]

['astronomia é uma ciência natural que estuda corpos celestes como estrelas, planetas, cometas, nebulosas, aglomerados de estrelas, galáxias e fenômenos que se originam fora da atmosfera da terra como a radiação cósmica de fundo em micro - ondas.',
 'preocupada com a evolução, a física, a química e o movimento de objetos celestes, bem como a formação e o desenvolvimento do universo.',
 'a astronomia é uma das mais antigas ciências.',
 'culturas pré - históricas deixaram registrados vários artefatos astronômicos, como stonehenge, os montes de newgrange e os menires.',
 'as primeiras civilizações, como os babilônios, gregos, chineses, indianos, iranianos e maias realizaram observações metódicas do céu noturno.',
 'no entanto, a invenção do telescópio permitiu o desenvolvimento da astronomia moderna.',
 'historicamente, a astronomia incluiu disciplinas tão diversas como astrometria, navegação astronômica, astronomia observacional e a elaboração de calendários.',
 'durante o período mediev

In [0]:
print(corp_list[30])
print(bert_tokenizer.tokenize(corp_list[30]))

omar khayyam ghiyath al - din abu 'l - fath umar ibn ibrahim al - nisaburi al - khayyami foi um grande cientista, filósofo e poeta persa que viveu de 1048 a 1131. compilou muitas tabelas astronômicas e executou uma reforma do calendário que era mais exato do que o calendário juliano e se aproximava do calendário gregoriano.
['o', 'mar', 'kha', 'y', 'y', 'am', 'g', 'hi', 'y', 'ath', 'al', '-', 'din', 'abu', "'", 'l', '-', 'fa', 'th', 'uma', 'r', 'i', 'bn', 'i', 'bra', 'h', 'im', 'al', '-', 'nis', 'abu', 'ri', 'al', '-', 'kha', 'y', 'y', 'ami', 'foi', 'um', 'grande', 'cientista', ',', 'filósofo', 'e', 'poeta', 'persa', 'que', 'viveu', 'de', '10', '48', 'a', '113', '1', '.', 'compil', 'ou', 'muitas', 'ta', 'belas', 'astro', 'nô', 'micas', 'e', 'execu', 'tou', 'uma', 'reforma', 'do', 'calendário', 'que', 'era', 'mais', 'ex', 'ato', 'do', 'que', 'o', 'calendário', 'juli', 'ano', 'e', 'se', 'aproxima', 'va', 'do', 'calendário', 'greg', 'ori', 'ano', '.']


In [0]:
TRAIN_VAL_SPLIT = .1
train_size = int(len(corp_list)*TRAIN_VAL_SPLIT)

df_train = pd.DataFrame(corp_list[:train_size])
df_val = pd.DataFrame(corp_list[train_size:])
df_val.to_csv(processed_data_dir / "validate" / "val.csv", index=False, header=None, quoting=csv.QUOTE_MINIMAL, encoding='utf-8')

step = int(train_size/100)
for i in range(100):
    df_train[step*i:step*(i+1)].to_csv(processed_data_dir / "train" / "train{}.csv".format(i), index=False, header=None, quoting=csv.QUOTE_MINIMAL, encoding='utf-8')